In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nltk

# Lenguaje Claro - UPE
Lucas Pecina

09/01/21

---
### Objetivo
Comparar distintos textos (legales, judiciales, etc) para ver la complejidad que tiene ese texto en cuanto a palabras dificiles (poco frecuentes en el lenguaje cotidiano).

### Resumen
El lenguaje legal/judicial suele ser complicado de entender para la gente que no se dedica a temas relacionados debido a la complejidad de las palabras utilizadas y de su gramatica. El analisis de complejidad gramatical de los textos requiere la utilizacion de tecnicas muy sofisticadas y que no dan buenos resultados, por lo tanto, descartamos este tipo de estudio por el momento.

Comenzamos solo analizando las **palabras** de los textos y creamos un **indice de complejidad** que mide la cantidad y frecuencia de palabras "complejas", que son de poca frecuencia en el lenguaje cotidiano. 

No contaran para el calculo aquellas palabras que por su naturaleza tecnica juridica requiera de su uso obligatorio. Para dicha tarea, se elaborara a mano una lista de las palabras en cuestion.

### Fuentes de datos
- Textos a analizar: seran cargados como archivos de texto
 - Codigo Procesal Civil y Comercial actual http://biblioteca.asesoria.gba.gov.ar/redirect.php?id=2121 esta en un archivo CPCC
 - Codigo Procesal Penal actual http://servicios.infoleg.gob.ar/infolegInternet/anexos/0-4999/383/texact.htm 
- Bolsa de palabras de frecuencia del lenguaje español http://corpus.rae.es/lfrecuencias.html. Usando las 300mil palabras mas frecuentes del español: en datos/

### Pasos
1. Corpus de palabras frecuentes
 - Armar dataframe con frecuencias
 - Calcular complejidad de palabras
2. Cargar textos y preprocesarlo
 - Tokenizar las palabras
 - Limpiar errores
 - Remover stopwords


---
### Indice de complejidad del lenguaje
$$Complejidad_t = \frac{1}{P}\sum_p^P{Complejidad_p}$$

- t = texto a analizar
- p = lista de palabras en el texto t (pueden estar repetidas)
- P = cantidad de palabras en el texto t (todas)

$$Complejidad_p = f(FrecuenciaBolsa_p)$$

$$Complejidad_p = \frac{1}{FrecuenciaBolsa_p}$$


Corpus de palabras frecuentes del español
---



In [175]:
# cargo datos
corpus = pd.read_excel('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CREA_total.xlsx')
corpus = corpus.dropna()
corpus['Palabra'] = corpus.Palabra.astype('str').str.strip()
corpus = corpus[corpus.Frecuencia_norm>0.00]

# reemplazo caracteres mal cargados
reemplazar = {'·':'á','È':'é','Ò':'ñ','˙':'ú','¸':'ü','Ì':'í','Û':'ó'}

for k,v in reemplazar.items():
    corpus['Palabra'] = corpus.Palabra.str.replace(k,v)
corpus

,Orden,Palabra,Frecuencia,Frecuencia_norm
0,1,de,9999518,65545.55
1,2,la,6277560,41148.59
2,3,que,4681839,30688.85
3,4,el,4569652,29953.48
4,5,en,4234281,27755.16
...,...,...,...,...
423729,423730,zyscovich,2,0.01
423730,423731,zyx,2,0.01
423731,423732,zzz,2,0.01
423732,423733,zzzzzzzzzzz,2,0.01


In [176]:
corpus.Frecuencia_norm.sum()

995706.050000001

In [177]:
# calculo complejidad
corpus['complejidad'] = 1/corpus.Frecuencia_norm
corpus

,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad
0,1,de,9999518,65545.55,0.000015
1,2,la,6277560,41148.59,0.000024
2,3,que,4681839,30688.85,0.000033
3,4,el,4569652,29953.48,0.000033
4,5,en,4234281,27755.16,0.000036
...,...,...,...,...,...
423729,423730,zyscovich,2,0.01,100.000000
423730,423731,zyx,2,0.01,100.000000
423731,423732,zzz,2,0.01,100.000000
423732,423733,zzzzzzzzzzz,2,0.01,100.000000


Procesamiento de textos a analizar
---


In [186]:
# funcion para el preprocesamiento de los textos a cargar

def limpieza_textos(raw,corpus):
    tokens = nltk.word_tokenize(raw)
    tokens = [t.lower() for t in tokens if t.isalpha()]
    print(f'El texto original tiene {len(tokens)} palabras.')
    print(f'De esas, hay {len(np.unique(np.array(tokens)))} unicas')
    
    # hay muchos que finalizan en "are", 'ere'. Mencionare, juzgare, interpusiere, etc.
    # a esos los saco
    tokens = [t for t in tokens if ('are' not in t) & ('ere' not in t)]
    print(f'Despues de sacarle las palabras que terminan en ARE o ERE, quedan {len(tokens)} palabras.')
    print(f'Ahora hay {len(np.unique(np.array(tokens)))} unicas')
    
    # veo las palabras del texto que no estan en el corpus y las agrego al corpus para que cuenten
    no_estan = [t for t in np.unique(np.array(tokens)) if t not in corpus.Palabra.values]
    print(f'Hay {len(no_estan)} parabras unicas que no aparecen en el corpus. Las agrego con 100 de complejidad')
    
    # ahora quedaron solo las que son muy raras o las que estan sin tildes por los titulos
    '''Ahora hay que agregar todas estas palabras al df de corpus y agregarle
    el valor maximo = 100 '''

    no_estan_df = pd.DataFrame({'Palabra':no_estan, 'complejidad':[100.0]*len(no_estan)})
    print('no estan df')
    print(no_estan_df)
    # concat
    corpus_ag = pd.concat([corpus[['Palabra','complejidad']],no_estan_df]).reset_index()
    
    return(tokens,corpus_ag)    

In [301]:
# funcion para calcular frecuencias de palabras en texto y calculo el indice_complejidad

def crear_indice_complejidad(tokens,corpus,nombre_texto,comunes_sacar):
    '''1 crear df de palabras de nombre_texto y las frecuencias
       2 joinear y calcular'''

    tokens_freq = pd.DataFrame.from_dict(nltk.FreqDist(tokens), orient='index').reset_index()
    tokens_freq.columns = ['Palabra','freq']

    print('tokens_freq')
    print(tokens_freq)
    
    # joinear
    tokens_complejidad = corpus.iloc[comunes_sacar:,:].merge(tokens_freq, left_on='Palabra',right_on='Palabra').reset_index(drop=True)
    # outer joinear pero para ver los que no estan
    no_estan = corpus.iloc[comunes_sacar:,:].merge(tokens_freq, left_on='Palabra',right_on='Palabra',how='outer').reset_index(drop=True)
    no_estan = no_estan[pd.isna(no_estan.Frecuencia)]
    
    # calculo indice_complejidad
    tokens_complejidad['indice_complejidad'] = (tokens_complejidad.complejidad * tokens_complejidad.freq)/tokens_complejidad.freq.sum()
    print(f'la cantidad de palabras que coincidieron en el corpus es de {len(tokens_complejidad)}')
    
    indice_complejidad = round(sum(tokens_complejidad.indice_complejidad),5)
    print(f'INDICE DE COMPLEJIDAD DE {nombre_texto}: {indice_complejidad}')
    
    return(tokens_complejidad, indice_complejidad, no_estan)

In [277]:
# funcion para el preprocesamiento de los textos a cargar

def limpieza_textos_simple(raw, sacar_ere_ese=False):
    '''Considera como errores/cosas extremas a las palabras que estan
    en el texto pero no en el corpus'''
    tokens = nltk.word_tokenize(raw)
    tokens = [t.lower() for t in tokens if t.isalpha()]
    
    if sacar_ere_ese:
        # saco a los que finalizan en "are", 'ere'. Mencionare, juzgare, interpusiere, etc.
        tokens = [t for t in tokens if ('are' not in t) & ('ere' not in t)]
        print('Luego de sacar las que tienen ARE o ERE (como conociere)')
    
    print(f'El texto original tiene {len(tokens)} palabras.')
    print(f'De esas, hay {len(np.unique(np.array(tokens)))} unicas')

    return(tokens)

In [262]:
comunes_sacar = 0
sacar_ere_ese = False

CPCC
---

In [278]:
cpcc_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CPCC.txt')
cpcc_raw = cpcc_f.read()
cpcc_tokens = limpieza_textos_simple(cpcc_raw, sacar_ere_ese)
cpcc_tokens, cpcc_indice = crear_indice_complejidad(cpcc_tokens,corpus,'CPCC actual',comunes_sacar)
cpcc_tokens.sort_values('indice_complejidad',ascending=False).head(10)

El texto original tiene 62785 palabras.
De esas, hay 5627 unicas
tokens_freq
            Palabra  freq
0             libro    23
1                 i    32
2     disposiciones    45
3         generales    16
4            título    70
...             ...   ...
5622       cúmplase     1
5623    comuníquese     1
5624     publíquese     1
5625           dese     1
5626      archívese     1

[5627 rows x 2 columns]
la cantidad de palabras que coincidieron en el corpus es de 5359
INDICE DE COMPLEJIDAD DE CPCC actual: 0.81736


,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad,freq,indice_complejidad
5210,258917,correspondiere,4,0.02,50.0,27,0.021669
5303,350809,compareciere,2,0.01,100.0,13,0.020866
5257,297261,devolutivo,3,0.01,100.0,13,0.020866
5202,253597,apelable,4,0.02,50.0,23,0.018459
5232,267522,irrecurrible,4,0.02,50.0,13,0.010433
5273,310387,litispendencia,3,0.01,100.0,6,0.009631
5294,332839,admitiere,2,0.01,100.0,6,0.009631
5203,253598,apelante,4,0.02,50.0,10,0.008026
5336,392124,notificador,2,0.01,100.0,5,0.008026
5259,297747,dispusiere,3,0.01,100.0,5,0.008026


CPP
---

In [279]:
cpp_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CPP.txt')
cpp_raw = cpp_f.read()

cpp_tokens = limpieza_textos_simple(cpp_raw, sacar_ere_ese)
cpp_tokens, cpp_indice = crear_indice_complejidad(cpp_tokens,corpus,'CPP actual',comunes_sacar)
cpp_tokens.sort_values('indice_complejidad',ascending=False).head(10)

El texto original tiene 44432 palabras.
De esas, hay 4445 unicas
tokens_freq
           Palabra  freq
0           codigo    11
1         procesal    29
2            penal   101
3              ley   163
4              ver     2
...            ...   ...
4440  permanecerán     1
4441       entrará     1
4442     efectuada     1
4443       reforma     1
4444       órganos     1

[4445 rows x 2 columns]
la cantidad de palabras que coincidieron en el corpus es de 4357
INDICE DE COMPLEJIDAD DE CPP actual: 0.47422


,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad,freq,indice_complejidad
4340,381324,labrará,2,0.01,100.000000,7,0.015797
4329,326089,supiere,3,0.01,100.000000,7,0.015797
4285,253597,apelable,4,0.02,50.000000,11,0.012412
4336,350809,compareciere,2,0.01,100.000000,4,0.009027
4303,267522,irrecurrible,4,0.02,50.000000,7,0.007899
4292,258917,correspondiere,4,0.02,50.000000,7,0.007899
4349,401415,quáter,2,0.01,100.000000,3,0.006770
4334,332839,admitiere,2,0.01,100.000000,3,0.006770
4274,237599,excusación,5,0.03,33.333333,8,0.006018
4253,217807,encontrare,6,0.03,33.333333,7,0.005266


Comparacion
---

In [280]:
print(f'La tasa de complejidad de uno sobre el otro es de: \n{round(cpcc_indice/cpp_indice, 2)} ({round((cpcc_indice-cpp_indice)/cpp_indice*100, 1)}% mas complejo)')

La tasa de complejidad de uno sobre el otro es de: 
1.72 (72.4% mas complejo)


Prueba borges
---


In [283]:
borges_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/borges-senderos.txt')
borges_raw = borges_f.read()

borges_tokens = limpieza_textos_simple(borges_raw, sacar_ere_ese)
borges_tokens, borges_indice = crear_indice_complejidad(borges_tokens,corpus,'borges',comunes_sacar)
borges_tokens.sort_values('indice_complejidad',ascending=False).head(30)

El texto original tiene 3650 palabras.
De esas, hay 1310 unicas
tokens_freq
          Palabra  freq
0              en   102
1              la   132
2          página     3
3              de   215
4        historia     2
...           ...   ...
1305    estrépito     1
1306        hallé     1
1307        saber     1
1308  innumerable     1
1309   contrición     1

[1310 rows x 2 columns]
la cantidad de palabras que coincidieron en el corpus es de 1293
INDICE DE COMPLEJIDAD DE borges: 0.62843


,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad,freq,indice_complejidad
1292,392684,ochavados,2,0.01,100.000000,1,0.027663
1283,268983,lidell,4,0.02,50.000000,2,0.027663
1291,331837,acaballado,2,0.01,100.000000,1,0.027663
1290,327297,tientsin,3,0.01,100.000000,1,0.027663
1289,299383,enclaustró,3,0.01,100.000000,1,0.027663
1288,292955,coalescentes,3,0.01,100.000000,1,0.027663
1286,281038,tsun,4,0.02,50.000000,2,0.027663
1270,195314,porvenires,8,0.05,20.000000,4,0.022130
1244,107467,madden,27,0.17,5.882353,11,0.017899
1267,178162,argüí,9,0.05,20.000000,3,0.016598


Corazon delator - Poe
---

In [287]:
poe_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/corazon-delator.txt')
poe_raw = poe_f.read()

poe_tokens = limpieza_textos_simple(poe_raw, sacar_ere_ese)
poe_tokens, poe_indice = crear_indice_complejidad(poe_tokens,corpus,'poe - corazon delator',comunes_sacar)
poe_tokens.sort_values('indice_complejidad',ascending=False).head(10)

El texto original tiene 2059 palabras.
De esas, hay 744 unicas
tokens_freq
      Palabra  freq
0      cierto     1
1     siempre     3
2          he     3
3        sido     3
4    nervioso     4
..        ...   ...
739      otra     1
740    fingir     1
741  malvados     1
742      maté     1
743      esos     1

[744 rows x 2 columns]
la cantidad de palabras que coincidieron en el corpus es de 741
INDICE DE COMPLEJIDAD DE poe - corazon delator: 0.28718


,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad,freq,indice_complejidad
740,411080,sobrepujaba,2,0.01,100.000000,1,0.048638
739,323994,sentáronse,3,0.01,100.000000,1,0.048638
738,275700,raspé,4,0.02,50.000000,1,0.024319
737,248504,sobresaltara,5,0.03,33.333333,1,0.016213
736,236774,enfurecerme,5,0.03,33.333333,1,0.016213
735,164219,descansaran,11,0.07,14.285714,1,0.006948
734,163814,conduciéndolos,11,0.07,14.285714,1,0.006948
733,152435,erradamente,13,0.08,12.500000,1,0.006080
727,97564,revisaran,33,0.21,4.761905,2,0.004632
732,121482,insignificancias,21,0.13,7.692308,1,0.003741


Juli libro
---

In [302]:
juli_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/juli.txt')
juli_raw = juli_f.read()

juli_tokens = limpieza_textos_simple(juli_raw, sacar_ere_ese)
juli_tokens, juli_indice, juli_no_estan = crear_indice_complejidad(juli_tokens,corpus,'juli',comunes_sacar)
juli_tokens.sort_values('indice_complejidad',ascending=False).head(30)

El texto original tiene 2270 palabras.
De esas, hay 828 unicas
tokens_freq
        Palabra  freq
0    nuevamente     1
1            la    50
2        visión     4
3            se    42
4       evaporó     1
..          ...   ...
823   desganada     1
824      volver     1
825      corrió     1
826        tras     1
827   detenerla     1

[828 rows x 2 columns]
la cantidad de palabras que coincidieron en el corpus es de 815
INDICE DE COMPLEJIDAD DE juli: 0.39397


,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad,freq,indice_complejidad
814,340167,avior,2,0.01,100.000000,1,0.044964
813,284909,alucinador,3,0.01,100.000000,1,0.044964
773,67029,desmond,66,0.43,2.325581,24,0.025096
812,257028,causaste,4,0.02,50.000000,1,0.022482
806,198236,abraxas,7,0.04,25.000000,2,0.022482
802,159721,marcellus,12,0.07,14.285714,3,0.019270
811,229107,acechándolo,5,0.03,33.333333,1,0.014988
809,208570,provocarte,7,0.04,25.000000,1,0.011241
807,199324,asesinaste,7,0.04,25.000000,1,0.011241
810,209706,sauro,7,0.04,25.000000,1,0.011241


In [303]:
juli_no_estan

,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad,freq
423733,NaN,revolt,NaN,NaN,NaN,3.0
423734,NaN,servello,NaN,NaN,NaN,11.0
423735,NaN,parlam,NaN,NaN,NaN,1.0
423736,NaN,astrav,NaN,NaN,NaN,1.0
423737,NaN,azha,NaN,NaN,NaN,2.0
423738,NaN,reinad,NaN,NaN,NaN,1.0
423739,NaN,mourgatz,NaN,NaN,NaN,4.0
423740,NaN,interesaste,NaN,NaN,NaN,1.0
423741,NaN,cromiris,NaN,NaN,NaN,3.0
423742,NaN,zoá,NaN,NaN,NaN,16.0


king - el instituto
---

In [293]:
king_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/king.txt')
king_raw = king_f.read()

king_tokens = limpieza_textos_simple(king_raw, sacar_ere_ese)
king_tokens, king_indice = crear_indice_complejidad(king_tokens,corpus,'king',comunes_sacar)
king_tokens.sort_values('indice_complejidad',ascending=False).head(30)

El texto original tiene 2129 palabras.
De esas, hay 907 unicas
tokens_freq
        Palabra  freq
0       treinta     1
1       minutos     2
2       después     3
3            de   130
4            la    90
..          ...   ...
902     rústica     1
903      fumaba     1
904  cigarrillo     1
905       vista     1
906      saludó     1

[907 rows x 2 columns]
la cantidad de palabras que coincidieron en el corpus es de 896
INDICE DE COMPLEJIDAD DE king: 0.36334


,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad,freq,indice_complejidad
895,423213,zano,2,0.01,100.000000,1,0.047371
894,274717,prenderían,4,0.02,50.000000,1,0.023685
893,255140,basculan,4,0.02,50.000000,1,0.023685
883,147980,jamieson,14,0.09,11.111111,4,0.021054
892,225001,rácanos,6,0.03,33.333333,1,0.015790
881,134825,kellerman,17,0.11,9.090909,3,0.012919
891,194594,opinaran,8,0.05,20.000000,1,0.009474
890,192002,gomosas,8,0.05,20.000000,1,0.009474
889,178395,autorizasen,9,0.05,20.000000,1,0.009474
870,98052,brunswick,32,0.20,5.000000,3,0.007106
